# 03 · Backtest UPRO/SPXU Morning-Momentum Strategy

Run complete simulation of the duel strategy with PDT tracking, slippage, and deterministic execution.

**Goals**
- Load UPRO/SPXU simulated price data
- Run daily duel strategy loop (2010-present)
- Track equity, trades, PDT limits, and performance
- Visualize equity curve and trade statistics
- Calculate CAGR, Sharpe, max drawdown, MAR

In [ ]:
# Add parent directory to path for module imports
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from config import get_config
from momentum_lib import run_backtest, calculate_performance_stats

sns.set_theme(style="darkgrid")

config = get_config()
data_dir = Path("../data")

print("Modules loaded.")

In [ ]:
print("=" * 60)
print("LOADING PRICE DATA")
print("=" * 60)

# Load UPRO and SPXU price data
# Check if data files exist
upro_file = data_dir / "upro_prices.csv"
spxu_file = data_dir / "spxu_prices.csv"

if not upro_file.exists() or not spxu_file.exists():
    print("[SKIP] Data files not found. Please run 01_feature_engineering.ipynb first.")
    print(f"       Expected files: {upro_file} and {spxu_file}")
    print("=" * 60)
    raise FileNotFoundError("Data files not found. Run 01_feature_engineering.ipynb to generate data.")

upro_df = pd.read_csv(upro_file, parse_dates=["timestamp"], index_col="timestamp")
spxu_df = pd.read_csv(spxu_file, parse_dates=["timestamp"], index_col="timestamp")

# Ensure index is DatetimeIndex (handle timezone-aware strings)
if not isinstance(upro_df.index, pd.DatetimeIndex):
    upro_df.index = pd.to_datetime(upro_df.index, utc=True)
if not isinstance(spxu_df.index, pd.DatetimeIndex):
    spxu_df.index = pd.to_datetime(spxu_df.index, utc=True)

# Ensure timezone aware
if upro_df.index.tz is None:
    upro_df.index = upro_df.index.tz_localize("America/New_York")
else:
    upro_df.index = upro_df.index.tz_convert("America/New_York")
if spxu_df.index.tz is None:
    spxu_df.index = spxu_df.index.tz_localize("America/New_York")
else:
    spxu_df.index = spxu_df.index.tz_convert("America/New_York")

print(f"\n[OK] Data loaded")
print(f"     UPRO: {len(upro_df):,} rows")
print(f"     SPXU: {len(spxu_df):,} rows")
print(f"     Date range: {upro_df.index.min().date()} to {upro_df.index.max().date()}")
print("=" * 60)

upro_df.head()

In [ ]:
print("\n" + "=" * 60)
print("RUNNING BACKTEST")
print("=" * 60)
print(f"Start Capital:       ${config.start_capital:,.0f}")
print(f"Position Size:       {config.position_size_pct * 100:.1f}%")
print(f"Exit Mode:           {config.exit_mode}")
print(f"PDT Limits:          {'Enabled' if config.use_pdt_limits else 'Disabled'}")
print(f"Slippage:            {config.slippage_bps} bps")
print("=" * 60)

# Run the backtest
state, equity_curve, trades_df = run_backtest(upro_df, spxu_df, config)

print(f"\n[OK] Backtest complete")
print(f"     Total trades: {len(trades_df)}")
print(f"     Final equity: ${state.equity:,.2f}")
print(f"     Final cash: ${state.cash:,.2f}")
print("=" * 60)

In [ ]:
print("\n" + "=" * 60)
print("PERFORMANCE STATISTICS")
print("=" * 60)

stats = calculate_performance_stats(equity_curve)

print(f"Total Return:        {stats['total_return'] * 100:,.2f}%")
print(f"CAGR:                {stats['cagr'] * 100:,.2f}%")
print(f"Sharpe Ratio:        {stats['sharpe']:.2f}")
print(f"Max Drawdown:        {stats['max_drawdown'] * 100:,.2f}%")
print(f"Win Rate:            {stats['win_rate'] * 100:,.2f}%")
print(f"MAR Ratio:           {stats['mar']:.2f}")
print("=" * 60)

stats

In [ ]:
# Plot equity curve
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(equity_curve.index, equity_curve["equity"], linewidth=2)
ax.axhline(config.start_capital, color='gray', linestyle='--', alpha=0.5, label='Start Capital')
ax.set_title("Strategy Equity Curve", fontsize=16, fontweight='bold')
ax.set_xlabel("Date")
ax.set_ylabel("Equity ($)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze trades by role
print("=" * 60)
print("TRADE BREAKDOWN BY ROLE")
print("=" * 60)

role_counts = trades_df["role"].value_counts()
print(role_counts)
print()

# P&L by role
pnl_by_role = trades_df.groupby("role")["pnl"].agg(["sum", "mean", "count"])
print("\nP&L by Role:")
print(pnl_by_role)
print("=" * 60)

pnl_by_role

In [ ]:
# Save results
print("Saving results...")

equity_file = data_dir / "equity_curve.csv"
trades_file = data_dir / "trade_log.csv"

equity_curve.to_csv(equity_file)
trades_df.to_csv(trades_file, index=False)

print(f"[OK] Saved equity curve to {equity_file}")
print(f"[OK] Saved trade log to {trades_file}")

# Preview trade log
trades_df.head(20)